In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler, MCRotationMove, BaseIntegratorMove, IntegratorMoveError
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject
from openmmtools.integrators import *
import numpy as np
import copy, sys
import logging
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

In [2]:
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 1.0 * unit.femtoseconds
n_steps = 100

In [3]:
from openmmtools import testsystems
ala = testsystems.AlanineDipeptideVacuum()

In [4]:
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22))
alchemical_atoms = list(alchemical_region.alchemical_atoms)
alanine_alchemical_system = factory.create_alchemical_system(
    reference_system=ala.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [5]:
# Enslave lambda_sterics and lambda_electrostatics to a generic lambda variable.
alchemical_state.set_alchemical_variable('lambda', 0)
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

1.0
1.0


In [6]:
# The functions here turn off first electrostatic and the steric interactions
# in sequence as lambda goes from 1.0 to 0.0.
f_electrostatics = '2*(lambda-0.5)*step(lambda-0.5)'
f_sterics = '2*lambda*step_hm(0.5-lambda) + step_hm(lambda-0.5)'
alchemical_state.lambda_electrostatics = alchemy.AlchemicalFunction(f_electrostatics)
alchemical_state.lambda_sterics = alchemy.AlchemicalFunction(f_sterics)

alchemical_state.set_alchemical_variable('lambda', 0.75)
#print(dir(alchemical_state))
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

0.5
1.0


In [7]:
alchemical_state.set_alchemical_variable('lambda', 0.0)

In [8]:
thermo_state = ThermodynamicState(system=alanine_alchemical_system, temperature=300 * unit.kelvin)
compound_state = CompoundThermodynamicState(thermo_state, composable_states=[alchemical_state])

sampler_state = SamplerState(positions=ala.positions)

ncmc_integrator = AlchemicalNonequilibriumLangevinIntegrator(
    splitting='H R V O V R H',
    #splitting='O { V R H R V } O',
    temperature=temperature,
    collision_rate=collision_rate,
    timestep=timestep,
    nsteps_neq=n_steps,
    measure_heat=True)
integrator = LangevinIntegrator(
    temperature=temperature, timestep=timestep, collision_rate=collision_rate, measure_heat=True)

compound_integrator = CompoundIntegrator()
compound_integrator.addIntegrator(ncmc_integrator)
compound_integrator.addIntegrator(integrator)
compound_integrator.setCurrentIntegrator(0)

In [9]:
context_cache = cache.global_context_cache
context, compound_integrator = context_cache.get_context(compound_state, compound_integrator)

# If we reassign velocities, we can ignore the ones in sampler_state.
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(compound_state.temperature)

compound_integrator.step(2)

#print(dir(compound_state))
print(compound_state.lambda_sterics)
print(compound_state.lambda_electrostatics)

0.0
-0.0


In [10]:
#dir(compound_state)

In [11]:
# We get also velocities here even if we don't need them because we
# will recycle this State to update the sampler state object. This
# way we won't need a second call to Context.getState().
context_state = context.getState(
    getPositions=True, getVelocities=True, getEnergy=True, enforcePeriodicBox=compound_state.is_periodic)

# This is an optimization around the fact that Collective Variables are not a part of the State,
# but are a part of the Context. We do this call twice to minimize duplicating information fetched from
# the State.
# Update everything but the collective variables from the State object
sampler_state.update_from_context(context_state, ignore_collective_variables=True)
# Update only the collective variables from the Context
sampler_state.update_from_context(
    context, ignore_positions=True, ignore_velocities=True, ignore_collective_variables=False)

In [12]:
tmp_cmpd_integrator = context.getIntegrator()
integrator = tmp_cmpd_integrator.getIntegrator(0)
dir(integrator)

['BlockEnd',
 'ComputeGlobal',
 'ComputePerDof',
 'ComputeSum',
 'ConstrainPositions',
 'ConstrainVelocities',
 'IfBlockStart',
 'UpdateContextState',
 'WhileBlockStart',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__swig_getmethods__',
 '__swig_setmethods__',
 '__weakref__',
 '_s',
 'addComputeGlobal',
 'addComputePerDof',
 'addComputeSum',
 'addConstrainPositions',
 'addConstrainVelocities',
 'addGlobalVariable',
 'addPerDofVariable',
 'addTabulatedFunction',
 'addUpdateContextState',
 'beginIfBlock',
 'beginWhileBlock',
 'endBlock',
 'getComputationStep',
 'getConstraintTolerance',
 'get

In [13]:
ncmc_global_variables = {
            integrator.getGlobalVariableName(index): index for index in range(integrator.getNumGlobalVariables())
        }
print(ncmc_global_variables)

{'step': 12, 'heat': 4, 'b': 3, '_restorable__class_hash': 0, 'nsteps': 11, 'Eold': 8, 'lambda': 10, 'Enew': 9, 'a': 2, 'n_lambda_steps': 13, 'has_kT_changed': 15, 'lambda_step': 14, 'old_ke': 5, 'kT': 1, 'protocol_work': 7, 'new_ke': 6}


In [14]:
compound_state.set_alchemical_variable('lambda', ncmc_global_variables['lambda'])

In [15]:
print(compound_state.lambda_sterics)
print(compound_state.lambda_electrostatics)

1.0


AssertionError: lambda_electrostatics: 19.0